# TIDE COCO Example
This example downloads some Mask R-CNN results files from [detectron](https://github.com/facebookresearch/Detectron/blob/master/MODEL_ZOO.md) and then evaluates them using the COCO dataset.

### Grabbing Mask R-CNN results
For demonstration purposes, we'll be using Mask R-CNN results (Model ID 36229740), but you'll want to use your own results in practice.

In [1]:
# Download the Mask R-CNN results to test on. Only for demonstration purposes.
import urllib.request # For downloading the sample Mask R-CNN annotations

bbox_file = 'mask_rcnn_bbox.json'
mask_file = 'mask_rcnn_mask.json'

urllib.request.urlretrieve('https://dl.fbaipublicfiles.com/detectron/35861795/12_2017_baselines/e2e_mask_rcnn_R-101-FPN_1x.yaml.02_31_37.KqyEK4tT/output/test/coco_2014_minival/generalized_rcnn/bbox_coco_2014_minival_results.json', bbox_file)
urllib.request.urlretrieve('https://dl.fbaipublicfiles.com/detectron/35861795/12_2017_baselines/e2e_mask_rcnn_R-101-FPN_1x.yaml.02_31_37.KqyEK4tT/output/test/coco_2014_minival/generalized_rcnn/segmentations_coco_2014_minival_results.json', mask_file)

print('Results Downloaded!')

Results Downloaded!


### Running TIDE

In [2]:
# Import the TIDE evaluation toolkit
import sys
sys.path.append('..')
from tidecv import TIDE

# Import the datasets we want to use
import tidecv.datasets as datasets

In [3]:
# Load the dataset
# This will automatically download COCO's 2017 annotations for you. You can specify your own COCO-style dataset too!
# See datasets.py for a list of all the supported datasets.
gt = datasets.COCO()

In [4]:
# Load the results in COCO format
bbox_results = datasets.COCOResult(bbox_file) # These files were downloaded above.
mask_results = datasets.COCOResult(mask_file) # Replace them with your own in practice.

In [5]:
# Create a TIDE object to use for evaluation
tide = TIDE()

In [6]:
# Run the evaluations on the standard COCO metrics (i.e., a range of AP with IoU thresholds [50:95])
if False:
    tide.evaluate(gt, bbox_results) # Several options are available here, see the functions
    tide.evaluate(gt, mask_results) # evaluate and evaluate_range for more details.
else:
    tide.evaluate_multiple_models_on_one_gt(gt, [bbox_results, mask_results])
    tide.average_out_summary()

In [7]:
# Summarize the evaluations run so far in the console
tide.print_summary()

-- mask_rcnn_bbox --

mAP 50:95: 40.01
                                    Threshold AP @
  Thresh     50      55      60      65      70      75      80      85      90     95  
---------------------------------------------------------------------------------------
    AP     61.8   59.76   57.07   53.72   49.59   43.66   35.87   25.77   11.68   1.19  
                  Main Errors
  Type   Cls    Loc   Both   Dupe    Bkg   Miss  
------------------------------------------------
   dAP   3.4   6.65   1.18   0.19   3.96   7.53  
       Special Errors
  Type   FalsePos   FalseNeg  
-----------------------------
   dAP      16.28      15.57  
Precision: 21.16 | Recall: 78.41
AP (Small): 28.06 | AR (Small): 46.68
AP (Medium): 55.95 | AR (Medium): 73.7
AP (Large): 73.0 | AR (Large): 85.69

-- mask_rcnn_mask --

mAP 50:95: 35.92
                                    Threshold AP @
  Thresh     50      55      60      65      70      75      80      85     90     95  
-------------------------

In [8]:
# Save the summary to a JSON file
tide.save_summary('summary.json')

In [9]:
from tidecv.plotter import Plotter
import os

main_dir = os.getcwd()
out_dir = os.path.join(main_dir, "Plots")

summary_files = [f for f in os.listdir(main_dir) if f.startswith("summary") and f.endswith(".json") and not f.endswith("matrices.json")]

plotter = Plotter(summary_files=summary_files, output_dir=out_dir)
plotter.run()

All Plots created!
